In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers,models

In [3]:
#function to load and preprocess images
def load_and_preprocess_data(folder_path,label):
    images=[]
    labels=[]
    
    for filename in os.listdir(folder_path):
        img_path=os.path.join(folder_path,filename)
        img=cv2.imread(img_path)
        # Check if the image is not None
        if img is not None:
            img = cv2.resize(img, (64, 64))
            img = img / 255.0
            images.append(img)
            labels.append(label)
    return images,labels

In [4]:
traffic_labels={
    'accident':0,
    'dense_traffic':1,
    'fire':2,
    'sparse_traffic':3,
}

In [5]:
all_images=[]
all_labels=[]

In [6]:
for traffic,label in traffic_labels.items():
    folder_path=os.path.join('dataset',traffic)
    images,labels=load_and_preprocess_data(folder_path,label)
    all_images.extend(images)
    all_labels.extend(labels)
    
#convert to Numpy arrrays
all_images=np.array(all_images)
all_labels=np.array(all_labels)

In [7]:
all_labels

array([0, 0, 0, ..., 3, 3, 3])

In [8]:
all_images

array([[[[0.89019608, 0.82745098, 0.85490196],
         [0.28235294, 0.36078431, 0.61568627],
         [0.28235294, 0.34509804, 0.60392157],
         ...,
         [0.42745098, 0.38039216, 0.38823529],
         [0.28627451, 0.24313725, 0.2745098 ],
         [0.42352941, 0.4       , 0.41960784]],

        [[0.61568627, 0.60784314, 0.60392157],
         [0.47058824, 0.51764706, 0.62745098],
         [0.32156863, 0.35294118, 0.45490196],
         ...,
         [0.2745098 , 0.24313725, 0.24705882],
         [0.21960784, 0.18823529, 0.21568627],
         [0.2       , 0.17647059, 0.2       ]],

        [[0.49411765, 0.51764706, 0.54509804],
         [0.36862745, 0.37254902, 0.40784314],
         [0.25490196, 0.23921569, 0.25098039],
         ...,
         [0.17254902, 0.15294118, 0.14901961],
         [0.30588235, 0.2745098 , 0.30588235],
         [0.15686275, 0.13333333, 0.17647059]],

        ...,

        [[0.05490196, 0.07058824, 0.0745098 ],
         [0.07058824, 0.09019608, 0.08627451]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size = 0.2)

In [14]:
batch_size = 16
nb_epochs = 20
nb_filters = 32
nb_pool = 2
nb_conv = 3

In [15]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

In [16]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
for layer in base_model.layers:
    layer.trainable = False

In [17]:
# Create a new model on top
traffic_model_vgg16 = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])

In [18]:
traffic_model_vgg16.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
traffic_model_vgg16.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs, validation_data=(X_test, y_test))

Epoch 1/20
180/180 [==============================] - 19s 104ms/step - loss: 0.7000 - accuracy: 0.7285 - val_loss: 0.6278 - val_accuracy: 0.7583
Epoch 2/20
180/180 [==============================] - 20s 113ms/step - loss: 0.4625 - accuracy: 0.8299 - val_loss: 0.5737 - val_accuracy: 0.7764
Epoch 3/20
180/180 [==============================] - 22s 121ms/step - loss: 0.3732 - accuracy: 0.8608 - val_loss: 0.5382 - val_accuracy: 0.8208
Epoch 4/20
180/180 [==============================] - 21s 119ms/step - loss: 0.3072 - accuracy: 0.8910 - val_loss: 0.5781 - val_accuracy: 0.7917
Epoch 5/20
180/180 [==============================] - 22s 120ms/step - loss: 0.2693 - accuracy: 0.9062 - val_loss: 0.6113 - val_accuracy: 0.7931
Epoch 6/20
180/180 [==============================] - 23s 126ms/step - loss: 0.2168 - accuracy: 0.9233 - val_loss: 0.5858 - val_accuracy: 0.7986
Epoch 7/20
180/180 [==============================] - 25s 141ms/step - loss: 0.1925 - accuracy: 0.9278 - val_loss: 0.6776 - val_ac

In [19]:
traffic_model_vgg16.save('traffic_model_vgg16.h5')

C:\Users\shifz\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
from tensorflow.keras.models import load_model
loaded_model=load_model('traffic_model_vgg16.h5')

In [25]:
# Function to preprocess a test image
def preprocess_test_image(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (64, 64))
        img = img / 255.0
        return img
    else:
        return None

In [32]:
# Replace 'your_test_image.jpg' with the path to your test image
test_image_path = r"C:\Users\shifz\OneDrive\Desktop\traffic_system\dataset\images_016.jpg"
test_image = preprocess_test_image(test_image_path)

In [33]:
# Check if the image is not None
if test_image is not None:
    # Reshape the image to match the input shape expected by the model
    test_image = np.expand_dims(test_image, axis=0)
    # Make predictions
    predictions = traffic_model_vgg16.predict(test_image)

    # Map predicted class probabilities to class labels
    class_labels = {
        0: 'accident',
        1: 'dense_traffic',
        2: 'fire',
        3: 'sparse_traffic',
    }

    predicted_class = class_labels[np.argmax(predictions)]
    confidence = np.max(predictions)

    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence}")
else:
    print("Error: Unable to load and preprocess the test image.")

1/1 [==============================] - 0s 63ms/step
Predicted Class: sparse_traffic
Confidence: 0.9999477863311768
